In [ ]:
from appgeopy import *
from my_packages import *

#### Steps have been done (2024/9/3)
1. Extract point coordinates of GWL from HDF5 file --> to point shapefile
2. Create buffer from point shapefile (ArcGIS)
3. Get PointKey of points located within buffers
4. Using achieved PointKey, extract the points along with their cumulative displacements
5. Assign the GWL stations by which the points surround


In [ ]:
# include the cumulative displacement into the HDF5 file
point_gdf = pd.read_pickle(
    r"CDISP_in_GWL_buff500_byStation.pkl", compression="zip"
)
point_gdf.head(3)

In [ ]:
# extract the the InSAR datetime
insar_datetime = [col[1:] for col in point_gdf.columns if col.startswith("N")]

all_measurements = {"InSAR_datetime": insar_datetime}

new_metadata = {
    "Update001": "2024/08/28 : Add modeled GWL arrays",
    "Update002": "2024/08/31 : Add peaks/troughs derived from modeled GWL arrays",
    "Update003": "2024/09/03 : Add cumulative displacements of PS points within the 500m buffer of each GWL station",
}

available_stations = point_gdf["STATION"].unique()

for select_station in tqdm(available_stations):
    point_byStation = point_gdf.query("STATION==@select_station")
    pointkey = point_byStation["PointKey"].to_numpy(dtype="<S15")
    cdisp_df = point_byStation.loc[:, ["N" + col for col in insar_datetime]]
    cdisp_array = cdisp_df.to_numpy()
    all_measurements[select_station] = {
        "CDISP": cdisp_array,
        "PointKey": pointkey,
    }

In [ ]:
gwl_hdf5_file = "20240831_GWL_CRFP.h5"

# Copy the original HDF5 file to a secure version
shutil.copy2(src=gwl_hdf5_file, dst=gwl_hdf5_file.replace(".h5", "_secure.h5"))

# Extract existing data and metadata
with h5py.File(gwl_hdf5_file, "r") as hdf5_file:
    existing_data_dict = gwatertools.h5pytools.hdf5_to_data_dict(hdf5_file)
    existing_metadata_dict = gwatertools.h5pytools.hdf5_to_metadata_dict(
        hdf5_file
    )

# Update dictionaries with new measurement data
updated_data_dict = gwatertools.h5pytools.update_data_dict(
    existing_data_dict, all_measurements
)

# Update dictionaries with new measurement data
updated_metadata_dict = gwatertools.h5pytools.update_data_dict(
    existing_metadata_dict, new_metadata
)

# Write updated data and metadata back to a new HDF5 file
output_file_name = f"{datetime.now().strftime('%Y%m%d')}_GWL_CRFP.h5"
with h5py.File(output_file_name, "w") as hdf5_file:
    gwatertools.h5pytools.metadata_to_hdf5(hdf5_file, updated_metadata_dict)
    gwatertools.h5pytools.data_to_hdf5(hdf5_file, updated_data_dict)